In [2]:
import torch.nn as nn
import torchvision
from torchvision.models import resnet50, resnet18, squeezenet1_1
from torchvision.models.resnet import ResNet50_Weights, ResNet18_Weights
from torchvision.models.squeezenet import SqueezeNet1_1_Weights

class DEFmodel(nn.Module):
    def __init__(self):
        super(DEFmodel, self).__init__()
        self.pretrained = squeezenet1_1(weights = SqueezeNet1_1_Weights.DEFAULT)
        self.pretrained.fc = nn.Identity()

    def forward(self, x):
        x = self.pretrained(x)
        return x


In [18]:
import glob
import torch
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import numpy as np

data_dir = "/home/disi/Project-IML/gallery/"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gallery_paths = glob.glob(f"{data_dir}*", recursive=True)

model = DEFmodel()
model.to(device)

embeddings = {}


for path in gallery_paths:
    print(path)
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGRA2RGB)
    image = torchvision.transforms.ToTensor()(image)
    image =  torchvision.transforms.Resize((160, 160))(image)
    image = image.unsqueeze(0)
    image = image.to(device)
    embeddings[path] = model(image).cpu().detach().numpy()


/home/disi/Project-IML/gallery/100086_1.png


/home/disi/.virtualenvs/Project-IML/lib/python3.8/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


/home/disi/Project-IML/gallery/100121_0.png
/home/disi/Project-IML/gallery/100086_3.png
/home/disi/Project-IML/gallery/100108_1.png
/home/disi/Project-IML/gallery/100025_4.png
/home/disi/Project-IML/gallery/100129_0.png
/home/disi/Project-IML/gallery/100091_0.png
/home/disi/Project-IML/gallery/100162_4.png
/home/disi/Project-IML/gallery/100111_4.png
/home/disi/Project-IML/gallery/100149_2.png
/home/disi/Project-IML/gallery/100117_0.png
/home/disi/Project-IML/gallery/100075_4.png
/home/disi/Project-IML/gallery/100138_4.png
/home/disi/Project-IML/gallery/100144_1.png
/home/disi/Project-IML/gallery/100162_3.png
/home/disi/Project-IML/gallery/100083_0.png
/home/disi/Project-IML/gallery/100076_1.png
/home/disi/Project-IML/gallery/100151_2.png
/home/disi/Project-IML/gallery/100008_2.png
/home/disi/Project-IML/gallery/100134_1.png
/home/disi/Project-IML/gallery/100113_4.png
/home/disi/Project-IML/gallery/100066_3.png
/home/disi/Project-IML/gallery/100121_4.png
/home/disi/Project-IML/gallery/1

In [19]:
data_dir = "/home/disi/Project-IML/query/"
queries = {}
query_paths = glob.glob(f"{data_dir}*", recursive=True)
for query in query_paths:
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGRA2RGB)
    image = torchvision.transforms.ToTensor()(image)
    image =  torchvision.transforms.Resize((160, 160))(image)
    image = image.unsqueeze(0)
    image = image.to(device)
    queries[query] = model(image).detach().cpu().numpy()

for query in queries:
    query_embedding = queries[query]
    distances = {}
    for gallery in embeddings:
        gallery_embedding = embeddings[gallery].detach().cpu().numpy()
        distances[gallery] = euclidian_distance(query_embedding, gallery_embedding)
    sorted_distances = sorted(distances.items(), key=lambda x: x[1])
    print(f"Query: {query}")
    for i in range(5):
        print(f"Rank {i+1}: {sorted_distances[i][0]}")

Query: /home/disi/Project-IML/query/100086_1.png
Rank 1: /home/disi/Project-IML/gallery/100152_4.png
Rank 2: /home/disi/Project-IML/gallery/100113_1.png
Rank 3: /home/disi/Project-IML/gallery/100132_1.png
Rank 4: /home/disi/Project-IML/gallery/100024_4.png
Rank 5: /home/disi/Project-IML/gallery/100071_4.png
Query: /home/disi/Project-IML/query/100000_0.png
Rank 1: /home/disi/Project-IML/gallery/100152_4.png
Rank 2: /home/disi/Project-IML/gallery/100105_1.png
Rank 3: /home/disi/Project-IML/gallery/100045_3.png
Rank 4: /home/disi/Project-IML/gallery/100124_1.png
Rank 5: /home/disi/Project-IML/gallery/100024_4.png
Query: /home/disi/Project-IML/query/100067_4.png
Rank 1: /home/disi/Project-IML/gallery/100152_4.png
Rank 2: /home/disi/Project-IML/gallery/100113_1.png
Rank 3: /home/disi/Project-IML/gallery/100132_1.png
Rank 4: /home/disi/Project-IML/gallery/100124_1.png
Rank 5: /home/disi/Project-IML/gallery/100045_3.png
Query: /home/disi/Project-IML/query/100153_3.png
Rank 1: /home/disi/Proje

In [1]:
import os
import cv2
import numpy as np
import face_alignment

input_dir = "/home/disi/Project-IML/query/"
output_dir = "/home/disi/Project-IML/aligned_query/"

## default margin = 0.05
margin = 1

fa = face_alignment.FaceAlignment(face_alignment.LandmarksType._2D, flip_input=False, device="cuda")

for file_name in os.listdir(input_dir):
    if file_name.endswith('.jpg') or file_name.endswith('.png') or file_name.endswith('.jpeg'):
        img = cv2.imread(os.path.join(input_dir, file_name))
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        preds = fa.get_landmarks(gray)
        if preds is None:
            continue
        center = np.mean(preds[0], axis=0)
        left_eye = preds[0][36]
        right_eye = preds[0][45]
        dY = right_eye[1] - left_eye[1]
        dX = right_eye[0] - left_eye[0]
        angle = np.degrees(np.arctan2(dY, dX))
        M = cv2.getRotationMatrix2D(tuple(center), angle, 1)
        aligned_img = cv2.warpAffine(img, M, (img.shape[1], img.shape[0]))
        x, y, w, h = cv2.boundingRect(preds[0])
        h_margin = int(h*margin*2) if y - h*margin*2 > 0 else int(h*margin)

        ## debug ##
        w_margin = int(w*margin*2) if x - w*margin*2 > 0 else int(w*margin)
        aligned_img = aligned_img[y-h-int(h_margin):y+h+int(h_margin), x-w-int(w_margin):x+w+int(w_margin)]

        ##
        #aligned_img = aligned_img[y-int(h*margin):y+h+int(h*margin), x-int(w*margin):x+w+int(w*margin)]
        #print(aligned_img.shape)
        aligned_img = cv2.resize(aligned_img, (160, 160))
        cv2.imwrite(os.path.join(output_dir, file_name), aligned_img)


input_dir = "/home/disi/Project-IML/gallery/"
output_dir = "/home/disi/Project-IML/aligned_gallery/"

for file_name in os.listdir(input_dir):
    if file_name.endswith('.jpg') or file_name.endswith('.png') or file_name.endswith('.jpeg'):
        img = cv2.imread(os.path.join(input_dir, file_name))
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        preds = fa.get_landmarks(gray)
        if preds is None:
            continue
        if (len(preds) > 1):
            print("More than one face detected in image: ", file_name)

        center = np.mean(preds[0], axis=0)
        left_eye = preds[0][36]
        right_eye = preds[0][45]
        dY = right_eye[1] - left_eye[1]
        dX = right_eye[0] - left_eye[0]
        angle = np.degrees(np.arctan2(dY, dX))
        M = cv2.getRotationMatrix2D(tuple(center), angle, 1)
        aligned_img = cv2.warpAffine(img, M, (img.shape[1], img.shape[0]))
        x, y, w, h = cv2.boundingRect(preds[0])
        aligned_img = aligned_img[y-h-int(h*margin):y+h+int(h*margin), x-w-int(w*margin):x+w+int(w*margin)]
        try:
          aligned_img = cv2.resize(aligned_img, (160, 160))
        except:
          continue
        
        cv2.imwrite(os.path.join(output_dir, file_name), aligned_img)


/home/disi/.virtualenvs/Project-IML/lib/python3.8/site-packages/face_alignment/api.py:144: UserWarning: No faces were detected.
  warnings.warn("No faces were detected.")


More than one face detected in image:  100116_1.png


In [6]:
from PIL import Image
import torch
import glob

data_dir = "/home/disi/Project-IML/aligned_gallery/"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gallery_paths = glob.glob(f"{data_dir}*", recursive=True)

model = DEFmodel()
model.to(device)

embeddings = {}

for path in gallery_paths:
    image = Image.open(path)
    image = torchvision.transforms.ToTensor()(image)
    image =  torchvision.transforms.Resize((160, 160))(image)
    image = torchvision.transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))(image)
    image = image.unsqueeze(0)
    image = image.to(device)
    embeddings[path] = model(image).detach().cpu().numpy()
# save embeddings to disk
import pickle
with open("/home/disi/Project-IML/embeddings.pkl", "wb") as f:
    pickle.dump(embeddings, f)

data_dir = "/home/disi/Project-IML/aligned_query/"
queries = {}
query_paths = glob.glob(f"{data_dir}*", recursive=True)
for query in query_paths:
    image = Image.open(query)
    image = torchvision.transforms.ToTensor()(image)
    image =  torchvision.transforms.Resize((160, 160))(image)
    image = torchvision.transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))(image)
    image = image.unsqueeze(0)
    image = image.to(device)
    queries[query] = model(image).detach().cpu().numpy()

/home/disi/.virtualenvs/Project-IML/lib/python3.8/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [9]:
import numpy as np

def euclidian_distance(x, y):
    return np.sqrt(np.sum((x - y)**2))


for query in queries:
    query_embedding = queries[query]
    distances = {}
    for gallery in embeddings:
        gallery_embedding = embeddings[gallery]
        distances[gallery] = euclidian_distance(query_embedding, gallery_embedding)
    sorted_distances = sorted(distances.items(), key=lambda x: x[1])
    print(f"Query: {query}")
    print(sorted_distances[:5])

# now again but with another distance metric

from sklearn.metrics.pairwise import cosine_similarity

for query in queries:
    query_embedding = queries[query]
    distances = {}
    for gallery in embeddings:
        gallery_embedding = embeddings[gallery]
        distances[gallery] = cosine_similarity(query_embedding, gallery_embedding)[0][0]
    sorted_distances = sorted(distances.items(), key=lambda x: x[1], reverse=True)
    print(f"Query: {query}")
    print(sorted_distances[:5])

# set the number of retrieved images to display
num_retrievals = 5


Query: /home/disi/Project-IML/aligned_query/100086_1.png
[('/home/disi/Project-IML/aligned_gallery/100086_1.png', 35.28872), ('/home/disi/Project-IML/aligned_gallery/100094_1.png', 51.99954), ('/home/disi/Project-IML/aligned_gallery/100008_2.png', 56.503376), ('/home/disi/Project-IML/aligned_gallery/100162_0.png', 57.60647), ('/home/disi/Project-IML/aligned_gallery/100012_0.png', 60.201187)]
Query: /home/disi/Project-IML/aligned_query/100000_0.png
[('/home/disi/Project-IML/aligned_gallery/100000_0.png', 31.931337), ('/home/disi/Project-IML/aligned_gallery/100097_0.png', 61.141872), ('/home/disi/Project-IML/aligned_gallery/100119_4.png', 61.20998), ('/home/disi/Project-IML/aligned_gallery/100053_2.png', 62.034435), ('/home/disi/Project-IML/aligned_gallery/100006_4.png', 63.78065)]
Query: /home/disi/Project-IML/aligned_query/100067_4.png
[('/home/disi/Project-IML/aligned_gallery/100067_4.png', 37.725666), ('/home/disi/Project-IML/aligned_gallery/100137_1.png', 60.234818), ('/home/disi/Pr

In [ ]:
import matplotlib.pyplot as plt


# iterate over the queries
# set the number of retrieved images to display
num_retrievals = 5

# iterate over the queries
for query in queries:
    # get the query embedding and calculate distances
    query_embedding = queries[query]
    distances = {}
    for gallery in embeddings:
        gallery_embedding = embeddings[gallery]
        distances[gallery] = cosine_similarity(query_embedding, gallery_embedding)[0][0]

    # sort the distances in descending order
    sorted_distances = sorted(distances.items(), key=lambda x: x[1], reverse=True)

    # get the top retrieved image paths and distances
    top_paths = [x[0] for x in sorted_distances[:num_retrievals]]
    top_similarities = [x[1]*100 for x in sorted_distances[:num_retrievals]]

    # load the query image and the top retrieved images
    images = [Image.open(query)] + [Image.open(path) for path in top_paths]

    # create a figure with subplots for each image
    fig, axes = plt.subplots(1, num_retrievals+1, figsize=(15, 5))

    # display the query image
    axes[0].imshow(np.array(images[0]))
    axes[0].set_title("Query")

    # display the retrieved images and their distances
    for i in range(num_retrievals):
        axes[i+1].imshow(np.array(images[i+1]))
        axes[i+1].set_title(f"Similarity: {top_similarities[i]:.2f}%")
    plt.show()


# set the number of retrieved images to display

# iterate over the queries
for query in queries:
    # get the query embedding and calculate distances
    query_embedding = queries[query]
    distances = {}
    for gallery in embeddings:
        gallery_embedding = embeddings[gallery]
        distances[gallery] = euclidian_distance(query_embedding, gallery_embedding)

    # sort the distances in ascending order
    sorted_distances = sorted(distances.items(), key=lambda x: x[1])

    # get the top retrieved image paths and distances
    top_paths = [x[0] for x in sorted_distances[:num_retrievals]]
    top_distances = [x[1] for x in sorted_distances[:num_retrievals]]

    # load the query image and the top retrieved images
    images = [Image.open(query)] + [Image.open(path) for path in top_paths]

    # create a figure with subplots for each image
    fig, axes = plt.subplots(1, num_retrievals+1, figsize=(15, 5))

    # display the query image
    axes[0].imshow(np.array(images[0]))
    axes[0].set_title("Query")

    # display the retrieved images and their similarities
    for i in range(num_retrievals):
        axes[i+1].imshow(np.array(images[i+1]))
        axes[i+1].set_title(f"EuclidianD: {top_distances[i]:.2f}%")
    plt.show()



The cosine_similarity function returns a matrix of pairwise cosine similarities between the query embedding and the gallery embeddings. Since we are comparing one query image with multiple gallery images, we need to take the first element of the first dimension to get a single similarity score. Also, note that we pass reverse=True to sorted function to sort the distances in descending order since cosine similarity values range between -1 and 1, with higher values indicating more similarity.